# <center> Quantifying future local impacts of sea-level rise on buildings and infrastructure </center>
This notebook accompanies the manuscript, "Quantifying future local impacts of sea-level rise on buildings and infrastructure". The notebook and accompanying python scripts perform all analyses presented in the manuscript. To reduce clutter in this notebook, python code on the backend is called. This notebook is organized as follows:
+ [Methods](#methods) <br>
    + [Identify future water levels (temporal)](#step1)<br>
    + [Determine impacts of inundation on infrastructure (spatial)](#step2)<br>
    + [Map the timing of future water levels to impacts of inundation on infrastructure](#step3)<br>
+ [Plotting Results](#plotting)<br>



The following python packages are required:
<!-- | Stretch/Untouched |
| :- |
| [jupyter](https://jupyter.org/) |
| [matplotlib](https://matplotlib.org/) |
| [pandas](https://pandas.pydata.org/) |
| [geopandas](https://geopandas.org/en/stable/) |
| [numpy](https://numpy.org/) |
| [networkx](https://networkx.org/) |
| [pyincore](https://incore.ncsa.illinois.edu/) |
| [noaa_coops](https://github.com/GClunies/noaa_coops) | -->



+ [jupyter](https://jupyter.org/)
+ [matplotlib](https://matplotlib.org/)
+ [pandas](https://pandas.pydata.org/)
+ [geopandas](https://geopandas.org/en/stable/)
+ [numpy](https://numpy.org/)
+ [networkx](https://networkx.org/)
+ [pyincore](https://incore.ncsa.illinois.edu/)
+ [noaa_coops](https://github.com/GClunies/noaa_coops) <br>

Notebook and backend code written by **Dylan R. Sanderson** [dylan.sanderson@nist.gov](dylan.sanderson@nist.gov)

In [ ]:
import os
import matplotlib.pyplot as plt
import datetime

<a id='methods'></a>
## <center> Methods </center>
The figure below shows a detailed flowchart of the overall approach to assess future local impacts of SLR on buildings and infrastructure. 
There are three primary steps in the flowchart: (1) [identify future water levels (temporal)](#step1), (2) [determine impacts of inundation on infrastructure (spatial)](#step2), and (3) [map the timing of future water levels to impacts of inundation on infrastructure](#step3). This notebook is organized around these three steps.

<center><img src="figures/fig1-flowchart.png" width="70%"/></center>

<a id='step1'></a>
## <Center> Identify Future Water Levels (Temporal) </center>

The first step of the framework is to identify future water levels relative to a vertical reference datum. Total water levels at the shoreline can be defined as the sum of mean sea level, deterministic or astronomical tide, non-tidal residuals, and wave runup (Ruggiero et al., 2001). Depending on the community being considered appropriate aspects of total water level should be identified. For example, if a community is protected by a vertical sea wall, then considering wave runup may not be appropriate. When identifying the future water levels, the timeframe of analysis is also identified, e.g., 2025 to 2100. Multiple SLR scenarios should be identified in step 1 to account for uncertainty in future climate conditions. Local projections of SLR should be considered, as these can differ from global and regional projections due to factors such as vertical land motion (Sweet et al., 2022). 

This part of the code calls on NOAA's Center for Operational Oceanographic Products and Services (CO-OPS) API to retrieve the determinsitic tide. The SLR data was obtained from the US Army Corps of Engineers Sea Level Analysis Tool (https://climate.sec.usace.army.mil/slat/). This part of the code combines the SLR projections with the deterministic tide predictions. This python class is used in the subsequent analyses.

In [ ]:
from backend import SLR_Api
SLR = SLR_Api.SLR_API(
        station_id=8771450,                      # noaa tide station id
        scenario_names=['NOAA et al. 2022'],     # for SLR scenarios
        begin_date='20250101',                   # start date
        end_date='21001231',                     # end date
        load_tides=True,                         # include tides (true/false)
        nonexceendance_probs=[0.17, 0.5, 0.83]   # SLR percentiles
        )

This part of the code plots the intermediate SLR scenario with the tide prediction. The 50th percentile is shown.

In [ ]:
df = SLR.combined_df
xdata = SLR.combined_df.index
ydata_SLR = SLR.combined_df["SL_ft_MHHW_1.0_ne0.5"]*0.3048
ydata_SLR_wTide = SLR.combined_df["SL+Tide_ft_MHHW_1.0_ne0.5"]*0.3048
fig, ax = plt.subplots(1,1,figsize=(10,3))
ax.plot(xdata, ydata_SLR_wTide, lw=0.1, c='0.8')
ax.plot(xdata, ydata_SLR, lw=2, c='c')
ax.set_xlim([datetime.date(2025, 1, 1), datetime.date(2100, 1, 1)])
ax.set_title("Intermediate SLR Scenario - 50th percentile")
ax.set_ylabel("Water Level (m.; MHHW)")
ax.set_xlabel("Year")

<a id='step2'></a>
## <center> Determine Impacts of Inundation on Infrastructure (Spatial) </center>

The second step of the framework is to determine the impacts of SLR on infrastructure. This step considers the spatial extent of flooding resulting from increases in water level, but does not consider when this flooding will occur. Modeling the extent of flooding can range from static approximations to more complicated physics-based models. The former consists of bathtub models in which high water levels are overlaid on a digital elevation model to get water depths and inundation extents (e.g., NOAA, 2017). The latter consists of full physics-based models that capture hydrodynamic processes (e.g., Luettich et al., 1992). In between these two are reduced-physics models that solve simplified equations compared to their full physics-based counterparts (e.g., Leijnse et al., 2021). The framework shown in Fig. 1 can accommodate any of these approaches. <br>

<center><img src="figures/fig1-step2.png" width="60%"/></center>

Step 2 of the framework begins with considering an inundation footprint at 0 m. above the same vertical datum identified in Step 1. With this inundation footprint, the impacts to buildings, electric power, and transportation network are computed. Impacts to the building inventory are first determined by overlaying the inundation footprint on the building inventory. Every building in the building inventory is identified as being exposed to water or not. Exposure can be defined as either: (1) whether the water level exceeds the first-floor elevation, or (2) whether there is any standing water at the building’s location. These results are saved in a SLR impact database that will be used as a lookup table in Step 3. Impacts on the electric power system are then computed by overlaying the inundation footprint on the electric power system. Every building in the building inventory is assigned to a substation and it is assumed that if a substation is exposed, it cannot provide electricity to the buildings it services. These results are stored in the SLR impact database. Next, impacts to the transportation network are determined. The inundation footprint is first overlaid on the road and bridge transportation network to determine whether each road/bridge segment is exposed. The depth of inundation at each road segment is determined and results in either an increase in travel time along that segment, or the road segment becomes unpassable. Depth-disruption curves can be used to determine the maximum safe speed along each road segment (Pregnolato et al., 2017). Destinations in the road network are then identified and can be any point of interest in a community, such as grocery stores, libraries, schools, or medical facilities. The shortest path from every building to each destination are computed using the resulting travel times along each road segment. The travel times from each building are stored in the SLR impact database <br>



This process is then repeated until each inundation footprint is considered. For example, the next inundation footprint could be defined as 1ft. (0.3048 m.) above the vertical reference datum. Considering all inundation footprints results in a populated database that stores the impacts of increasing water levels on each building in the community. The database considers impacts for multiple infrastructure systems, e.g., for electric power the results are whether each building has electricity, whereas for transportation, the results are travel times from each building to a specific destination. The database can then be quickly queried, rather than having to run an inundation model and determine the impacts to infrastructure. 


### 2a. Impacts on Building Inventory
First, importing the building impact backend code. All functions to perform the impacts to the building inventory are in here. 

In [ ]:
from backend import ImpactsBuilding

This part of the code determines the impacts on the building inventory. This loops through each inundation footprint starting with 0 ft (0m) above MHHW and ending at 10 ft (3.048m) above MHHW. 

<font color='red'>Note that this part of the code takes a while to run. The outputs are provided in this repository, so this cell can be skipped if desired.</font>

In [ ]:
for slr_ft in range(0,11):
    print("----------------------")
    print("SLR: {}" .format(slr_ft))
    BD_SLR = ImpactsBuilding.BuildingExposureSLR()
    BD_SLR.RunBldgExposure(slr_ft=slr_ft)
    print()

This part of the code combines the results from the above into a single CSV file, or the "SLR impact database". This file is used as a lookup table in later steps. The results are saved in a CSV file called ```bldg-exp-combined.csv``` located in the ```output``` subdirectory.

In [ ]:
C = ImpactsBuilding.CombineBuildingExpSLR(slr_start=0, slr_end=10, save_df=True)


### 2b: Impacts on Electric Power System

Importing the eletric impact code.

In [ ]:
from backend import ImpactsElectric

This part of the code loops through each inundation layer and determines electricity loss for that layer. 

In [ ]:
for slr_ft in range(0,11):
    print("----------------------")
    print("SLR: {}" .format(slr_ft))
    EL_SLR = ImpactsElectric.electricity_access()
    EL_SLR.run_electricity_access(slr_ft=slr_ft)
    print()

This part of the code combines the results from above into a single CSV file, or the "SLR impact database". This file is used as a lookup table in later steps. The results are saved in a CSV file called ```elec-accs-combined.csv``` located in the ```output``` subdirectory.

In [ ]:
# # combining results
EL_SLR = ImpactsElectric.electricity_access()
df = EL_SLR.combine_elec_access()

### 2c: Impacts on Transportation Network

First importing the backend code for computing tranporation impacts.

In [ ]:
from backend import ImpactsTransportation

This part of the code loops through each inundation layer and determines transportation access for that layer. 

<font color='red'>Note that this part of the code takes a while to run. The outputs are provided in this repository, so this cell can be skipped, if desired.</font>

In [ ]:
for slr_ft in range(0,11):
    print("----------------------")
    print("SLR: {}" .format(slr_ft))
    te = ImpactsTransportation.transportation_exposure()
    te.run_transportation_exposure(
                    slr_ft=slr_ft,
                    locl_hzrd=True
                    )
    ta = ImpactsTransportation.transportation_access()
    ta.run_transportation_access(slr_ft=slr_ft, runname="utmb-hospital")
    ta.run_transportation_access(slr_ft=slr_ft, runname="galveston-exit")
    print()


In [ ]:
ta = ImpactsTransportation.transportation_access()
ta.combine_trns_access("utmb-hospital")
ta.combine_trns_access("galveston-exit")

<a id='step3'></a>
## <center> Map Future Water Levels to Impacts of Inundation on Infrastructure </center>

The final step of the framework is to map the future water levels from step 1 (temporal) to the impacts of inundation on infrastructure from step 2 (spatial). This step begins by considering the first future water level scenario that was identified in Step 1. The start year is initialized and daily high-water levels for the entire year are determined. For each daily high-water level, the impacts to infrastructure from the SLR impact database are queried. This is done by matching the daily high-water level to the nearest inundation footprint that was used in Step 2. The importance of using the same vertical reference datum is emphasized. This process repeats for each year of the future water level record, and then repeats again for each future water level scenario. Step 3 results in community-wide impacts of SLR on multiple infrastructure systems at the building level. By mapping the future water levels to the inundation footprints, both the timing and spatial extent of future impacts from SLR can be evaluated.

<center><img src="figures/fig1-step3.png" width="70%"/></center>


Importing the backend code to mape the water levels to the impacts to infrastructure

In [ ]:
from backend import MapWaterLevelsToImpacts
MPW = MapWaterLevelsToImpacts.MapWaterLevels(begindate_str = "20250101",
                                             enddate_str = "21001231", 
                                             station_id=8771450, 
                                             nonexceendance_probs=[0.17, 0.5, 0.83], 
                                             destination_points=["utmb-hospital", "galveston-exit"],
                                            )

Mapping the water levels to the building, electric, and transportation impacts. The outputs are stored in a subdirectory located as ```output/impacts-time```.

In [ ]:
MPW.map_bldg_impacts()
MPW.map_elec_impacts()
MPW.map_trns_impacts()

<a id='plotting'></a>
## <center> Plotting Results </center>
This section of the notebook plots the results of the above analyses. The following are selected to plot spatially:
+ A SLR scenario (low, intermediate-low, intermediate, intermediate-high, or high).
+ SLR scenario percentile (0.17, 0.5, and 0.83)
+ Infrastructure system (buildings, electric, transportation-exit, transportation-hospital)

In [ ]:
import pandas as pd
import functools
import geopandas as gpd
import ipywidgets as widgets
import os
from IPython.display import clear_output

In [ ]:
path_to_bldg_inv = os.path.join(os.getcwd(), "backend", "infrastructure", 'bldgs_drs.json')
bldgs = gpd.read_file(path_to_bldg_inv)
bldgs.to_crs(epsg=4269, inplace=True)
bldgs.set_index("guid", inplace=True)

In [ ]:
def generate_map(b, bldgs):
    output.clear_output()
    # clear_output(wait=True)
    inf_output_name = {"Buildings": "nTimesExp",
                       "Electric": "nNoAccess",
                       "Transportation-Exit": "nTTIncrease",
                       "Transportation-UTMB": "nTTIncrease"
                      }
    destination_mapping = {
                        "Transportation-Exit": "galveston-exit",
                        "Transportation-UTMB": "utmb-hospital",
                    }
    with output:
        print(slr_scenario_wi.value, percentile_wi.value, year_wi.value)
        path_to_file_results = os.path.join(os.getcwd(), "backend", "output", "impacts-time")
        if "Transportation" in inf_wi.value:
            fn = "{}_years_sc{}_ne{}_{}.csv" .format(inf_output_name[inf_wi.value], slr_scenario_wi.value, percentile_wi.value, destination_mapping[inf_wi.value])
        else:
            fn = "{}_years_sc{}_ne{}.csv" .format(inf_output_name[inf_wi.value], slr_scenario_wi.value, percentile_wi.value)
        fn = os.path.join(path_to_file_results, fn)
        df = pd.read_csv(fn)
        df.set_index("guid", inplace=True)
        bldgs = pd.merge(bldgs["geometry"], df[str(year_wi.value)], left_index=True, right_index=True)
        
        
        display(bldgs.explore(
                        column=str(year_wi.value),
                        cmap="RdYlGn_r",
                        vmin=0,
                        vmax=365,
                        # tiles="CartoDB dark_matter",
                        tiles="CartoDB positron",    
                        legend_kwds={"caption": "Days in {}" .format(year_wi.value)}
        ))

In [ ]:
output = widgets.Output()

button_wi = widgets.Button(
    description='Generate Map',
    disabled=False,
    button_style='info',
    tooltip='Generate Map',
    icon='check'
    )

slr_scenario_wi = widgets.Dropdown(
    options=["Low","Int-Low", "Int", "IntHigh", "High"],
    value='Int',
    description='SLR Scenario:',
    disabled=False,
)

percentile_wi = widgets.Dropdown(
    options=["0.17","0.5", "0.83"],
    value='0.5',
    description='Percentile',
    disabled=False,
)

year_wi = widgets.IntSlider(
    value=7,
    min=2025,
    max=2100,
    step=1,
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

inf_wi = widgets.Dropdown(
    options=["Buildings", "Electric", "Transportation-Exit", "Transportation-UTMB"],
    value='Buildings',
    description='Infr:',
    disabled=False,
)
sidebyside = widgets.HBox([slr_scenario_wi, percentile_wi, inf_wi])

Creating the map. <br>
<font color='red'>Note that sometimes the "Generate Map" button needs to be pressed twice for the map to generate. </font>

In [ ]:
display(sidebyside)
display(year_wi)
display(button_wi, output)
button_wi.on_click(functools.partial(generate_map, bldgs=bldgs))

## References
+ Ruggiero, P., Komar, P., McDougal, W., Marra, J., and Beach, R. (2001). Wave runup, extreme water levels and the erosion of properties backing beach. *Journal of Coastal Research*, 17(2):407-419. https://www.jstor.org/stable/4300192
  
+ Leijnse, T., van Ormondt, M., Nederhoff, K., and van Dongeren, A. (2021). Modeling compound flooding in coastal systems using a computationally efficient reduced-physics solver: Including fluvial, pluvial, tidal, wind- and wave-driven processes. Coastal Engineering, 163. https://doi.org/10.1016/j.coastaleng.2020.103796
  
+ Luettich, R., Westerink, J., and Scheffner, N. (1992). ADCIRC: An advanced three-dimensional circulation model for shelves, coasts, and estuaries, Report 1: Theory and Methodology of ADCIRC-2DDI and ADCIRC-3DL. US Army Engineer Waterways Experiment Station, TR DRP-92-6. https://hdl.handle.net/11681/4618
  
+ National Oceanic and Atmospheric Administration (NOAA) (2017). Detailed Method for Mapping Sea Level Rise Inundation. Office for Coastal Management, NOAA, US Department of Commerce. https://coast.noaa.gov/data/digitalcoast/pdf/slr-inundation-methods.pdf

+ Pregnolato, M., Ford, A., Wilkinson, S., and Dawson, R. (2017). The impact of flooding on road transport: A depth-disruption function. Transportation Research Part D, 55:67-81. https://doi.org/10.1016/j.trd.2017.06.020
 
+ Sweet, W., Hamlington, B., Kopp, R., Weaver, C., Barnard, P., Bekaert, D., Brooks, W., Craghan, M., Dusek, G., Frederikse, T., Garner, G., Genz, A., Krasting, J., Larour, E., Marcy, D., Marra, J., Obeysekera, J., Osler, M., Pendleton, M., Roman, D., Schmied, L., Veatch, W., White, K., and Zuzak, C. (2022). Global and regional sea level rise scenarios for the United States: Updated mean projections and extreme water level probabilities along U.S. coastlines. *NOAA Technical Report, NOS 01*. National Oceanic and Atmostpheric Administration, US Department of Commerce. https://oceanservice.noaa.gov/hazards/sealevelrise/noaa-nos-techrpt01-global-regional-SLR-scenarios-US.pdf